In [ ]:
!unzip /content/archive.zip -d /content/

Archive:  /content/archive.zip
replace /content/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
# ─── 0) Install prerequisites (Colab only) ─────────────────────────────────────
# !pip install requests scipy

import os
import zipfile
import requests
import numpy as np
import pandas as pd
from scipy.fft import rfft
import torch

# ─── 1) Download & unzip UCI HAR raw signals ───────────────────────────────────
UCI_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip"
ZIP_PATH  = "UCI_HAR.zip"
DATA_DIR  = "UCI_HAR_Dataset"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if not os.path.exists(DATA_DIR):
    print("Downloading raw UCI HAR dataset…")
    r = requests.get(UCI_URL)
    with open(ZIP_PATH, "wb") as f:
        f.write(r.content)
    with zipfile.ZipFile(ZIP_PATH, "r") as z:
        z.extractall()      # creates folder 'UCI HAR Dataset'
    os.rename("UCI HAR Dataset", DATA_DIR)

# ─── 2) Load your feature CSVs (563 features) ─────────────────────────────────
train_df = pd.read_csv("./train.csv")  # adjust path if needed
test_df  = pd.read_csv("./test.csv")

# ─── 3) Identify your 100 reduced features ────────────────────────────────────
# (If you have an explicit list, plug it in here; else auto‐select the first 100 non‐label cols)
all_feats = [c for c in train_df.columns if c != "Activity"]
reduced_feats = all_feats[:100]  # replace this with your actual 100‐feature list

# ─── 4) Load raw inertial signals into numpy arrays ───────────────────────────
def load_raw_windows(split="train"):
    base = os.path.join(DATA_DIR, split, "Inertial Signals")
    mats = []
    for signal in ["total_acc", "body_acc", "body_gyro"]:
        for axis in ["x","y","z"]:
            fn = f"{signal}_{axis}_{split}.txt"
            path = os.path.join(base, fn)
            arr  = pd.read_csv(path, delim_whitespace=True, header=None).values
            mats.append(arr[..., np.newaxis])
    # result: (n_windows, 128, 6)
    return np.concatenate(mats, axis=2)

raw_train = load_raw_windows("train")
raw_test  = load_raw_windows("test")

# ─── 5) Compute FFT features (20 bins × 6 channels = 120 dims) ────────────────
def compute_fft_features(X, n_bins=20, drop_dc=True):
    # X: (n_windows, 128, 6)
    fft_vals = np.abs(rfft(X, axis=1))       # → (n_windows, 65, 6)
    if drop_dc:
        fft_vals = fft_vals[:, 1:n_bins+1, :] # drop DC term
    else:
        fft_vals = fft_vals[:, :n_bins, :]
    return fft_vals.reshape(X.shape[0], -1)   # → (n_windows, 6*n_bins)

fft_train = compute_fft_features(raw_train, n_bins=20)
fft_test  = compute_fft_features(raw_test,  n_bins=20)

# ─── 6) Merge FFT + your 100 reduced features ─────────────────────────────────
# Extract labels
y_train = train_df["Activity"].values
y_test  = test_df["Activity"].values

# Build final feature matrices
X_train = np.hstack([train_df[reduced_feats].values, fft_train])  # shape (7352, 100+120)
X_test  = np.hstack([ test_df[ reduced_feats].values, fft_test ])  # shape (2947, 100+120)

print("X_train shape:", X_train.shape)
print("X_test  shape:", X_test.shape)


<ipython-input-8-8727767901c9>:43: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  arr  = pd.read_csv(path, delim_whitespace=True, header=None).values
<ipython-input-8-8727767901c9>:43: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  arr  = pd.read_csv(path, delim_whitespace=True, header=None).values
<ipython-input-8-8727767901c9>:43: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  arr  = pd.read_csv(path, delim_whitespace=True, header=None).values
<ipython-input-8-8727767901c9>:43: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  arr  = pd.read_csv(path, delim_whitespace=True, header=None).values
<ipython-inp

X_train shape: (7352, 280)
X_test  shape: (2947, 280)


In [ ]:
# Recommended Models for FFT Features + Example Training Code

import time
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assumes X_train, X_test, y_train, y_test are already defined in your notebook

models = {
    "SVM (RBF kernel)": SVC(kernel="rbf", C=1.0, gamma="scale"),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42),
    "MLP (Feedforward NN)": MLPClassifier(hidden_layer_sizes=(128, 64),
                                          activation="relu",
                                          solver="adam",
                                          max_iter=200,
                                          random_state=42)
}

results = []
for name, clf in models.items():
    print(f"\n=== Training & Evaluating: {name} ===")

    # Training
    start_train = time.time()
    clf.fit(X_train, y_train)
    train_time = time.time() - start_train

    # Inference
    start_test = time.time()
    y_pred = clf.predict(X_test)
    test_time = time.time() - start_test

    # Metrics
    acc = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {acc:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, zero_division=0))

    # Optional: Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:\n", cm)

    print(f"Training time: {train_time:.2f}s   Inference time: {test_time:.2f}s")
    print("-" * 50)

    results.append((name, acc, train_time, test_time))

# Summary table
import pandas as pd
summary = pd.DataFrame(results, columns=["Model", "Accuracy", "Train Time (s)", "Inference Time (s)"])
print("\n=== Summary ===")
print(summary)



=== Training & Evaluating: SVM (RBF kernel) ===
Accuracy: 0.9393
Classification Report:
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.89      0.84      0.86       491
          STANDING       0.86      0.90      0.88       532
           WALKING       0.99      0.99      0.99       496
WALKING_DOWNSTAIRS       0.97      0.92      0.94       420
  WALKING_UPSTAIRS       0.94      0.98      0.96       471

          accuracy                           0.94      2947
         macro avg       0.94      0.94      0.94      2947
      weighted avg       0.94      0.94      0.94      2947

Confusion Matrix:
 [[537   0   0   0   0   0]
 [  1 413  77   0   0   0]
 [  0  53 479   0   0   0]
 [  0   0   0 489   7   0]
 [  0   0   0   3 387  30]
 [  0   0   0   1   7 463]]
Training time: 1.97s   Inference time: 3.19s
--------------------------------------------------

=== Training & Evaluating: Ra

PINNs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader

# ─── Assumes the following are already defined in your notebook:
# raw_train (numpy array, shape [n_windows,128,6])
# raw_test  (numpy array, shape [n_windows,128,6])
# train_df, test_df with 'Activity' column
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Convert labels if not already done
le = LabelEncoder()
y_train_int = le.fit_transform(train_df['Activity'])
y_test_int  = le.transform(test_df['Activity'])

# Create PyTorch datasets
X_tr = torch.tensor(raw_train, dtype=torch.float32)
y_tr = torch.tensor(y_train_int, dtype=torch.long)
X_te = torch.tensor(raw_test,  dtype=torch.float32)
y_te = torch.tensor(y_test_int,  dtype=torch.long)

train_ds = TensorDataset(X_tr, y_tr)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_ds  = TensorDataset(X_te, y_te)
test_loader  = DataLoader(test_ds, batch_size=64)

# … after DataLoader setup …

# 3) Define PINN_HAR as before
class PINN_HAR(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=6):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, bidirectional=True)
        self.fc_cls  = nn.Linear(2*hidden_size, num_classes)
        self.fc_phys = nn.Linear(2*hidden_size, input_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        cls_logits = self.fc_cls(out[:, -1, :])
        phys_pred  = self.fc_phys(out)
        return cls_logits, phys_pred

# 4) Instantiate model with the correct channel dimension
n_channels = raw_train.shape[2]   # should be 9
n_classes  = len(le.classes_)     # should be 6 activities
model = PINN_HAR(input_size=n_channels,
                 hidden_size=64,
                 num_layers=2,
                 num_classes=n_classes).to(device)

# … continue with optimizer, loss definitions, training loop, etc. …


# # Instantiate model, optimizer, losses
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = PINN_HAR(input_size=6).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
ce_loss = nn.CrossEntropyLoss()
mse_loss = nn.MSELoss()
lambda_phys = 0.1

# ─── 4) Training loop ─────────────────────────────────────────────────────────
num_epochs = 50
for epoch in range(1, num_epochs + 1):
    model.train()
    total_cls, total_phys = 0.0, 0.0

    for X_batch, y_batch in train_loader:
        Xb = X_batch.to(device)
        yb = y_batch.to(device)

        optimizer.zero_grad()
        logits, phys = model(Xb)

        # Classification loss
        loss_cls = ce_loss(logits, yb)

        # Physics loss: second derivative approx
        d2_phys = phys[:, 2:] - 2 * phys[:, 1:-1] + phys[:, :-2]  # (B,126,6)
        acc_mid = Xb[:, 1:-1, :]                                 # (B,126,6)
        loss_phys = mse_loss(d2_phys, acc_mid)

        # Total loss
        loss = loss_cls + lambda_phys * loss_phys
        loss.backward()
        optimizer.step()

        total_cls  += loss_cls.item()
        total_phys += loss_phys.item()

    avg_cls = total_cls / len(train_loader)
    avg_phys = total_phys / len(train_loader)
    print(f"Epoch {epoch:2d} | CE Loss: {avg_cls:.4f} | Phys Loss: {avg_phys:.4f}")

# ─── Evaluation ───────────────────────────────────────────────────────────────
model.eval()
correct = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        Xb = X_batch.to(device)
        logits, _ = model(Xb)
        preds = logits.argmax(dim=1).cpu()
        correct += (preds == y_batch).sum().item()

accuracy = correct / len(test_ds)
print(f"\nTest Accuracy (PINN): {accuracy:.4f}")


Epoch  1 | CE Loss: 1.3295 | Phys Loss: 0.1675
Epoch  2 | CE Loss: 0.9001 | Phys Loss: 0.1548
Epoch  3 | CE Loss: 0.7265 | Phys Loss: 0.1456
Epoch  4 | CE Loss: 0.6328 | Phys Loss: 0.1404
Epoch  5 | CE Loss: 0.5891 | Phys Loss: 0.1366
Epoch  6 | CE Loss: 0.4938 | Phys Loss: 0.1340
Epoch  7 | CE Loss: 0.3613 | Phys Loss: 0.1324
Epoch  8 | CE Loss: 0.2414 | Phys Loss: 0.1312
Epoch  9 | CE Loss: 0.1845 | Phys Loss: 0.1296
Epoch 10 | CE Loss: 0.1719 | Phys Loss: 0.1278
Epoch 11 | CE Loss: 0.1440 | Phys Loss: 0.1265
Epoch 12 | CE Loss: 0.1392 | Phys Loss: 0.1254
Epoch 13 | CE Loss: 0.1254 | Phys Loss: 0.1238
Epoch 14 | CE Loss: 0.1251 | Phys Loss: 0.1228
Epoch 15 | CE Loss: 0.1222 | Phys Loss: 0.1224
Epoch 16 | CE Loss: 0.1214 | Phys Loss: 0.1213
Epoch 17 | CE Loss: 0.1221 | Phys Loss: 0.1206
Epoch 18 | CE Loss: 0.1489 | Phys Loss: 0.1199
Epoch 19 | CE Loss: 0.1194 | Phys Loss: 0.1189
Epoch 20 | CE Loss: 0.1229 | Phys Loss: 0.1190
Epoch 21 | CE Loss: 0.1143 | Phys Loss: 0.1208
Epoch 22 | CE

# Transformers (use GPU)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder

# ─── 0) Prepare your data arrays ────────────────────────────────────────────
# raw_train, raw_test: np.arrays, shape (n_windows, 128, n_channels)
# train_df/test_df: pandas DataFrames with an 'Activity' column

# Label encoding
le = LabelEncoder()
y_train = le.fit_transform(train_df['Activity'].values)
y_test  = le.transform(    test_df['Activity'].values)

# TensorDatasets
X_tr = torch.tensor(raw_train, dtype=torch.float32)
y_tr = torch.tensor(y_train,    dtype=torch.long)
X_te = torch.tensor(raw_test,  dtype=torch.float32)
y_te = torch.tensor(y_test,    dtype=torch.long)

train_ds = TensorDataset(X_tr, y_tr)
test_ds  = TensorDataset(X_te, y_te)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=64)

# ─── 1) Positional Encoding ─────────────────────────────────────────────────
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=128):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2).float() *
                        (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.pe = pe.unsqueeze(0)  # shape (1, max_len, d_model)

    def forward(self, x):
        # x: (batch, seq_len, d_model)
        return x + self.pe[:, :x.size(1), :].to(x.device)

# ─── 2) Transformer Classifier ───────────────────────────────────────────────
class TransformerHAR(nn.Module):
    def __init__(self, n_channels, d_model=64, n_heads=4,
                 num_layers=2, num_classes=6, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(n_channels, d_model)
        self.pos_enc     = PositionalEncoding(d_model, max_len=128)
        encoder_layer    = nn.TransformerEncoderLayer(d_model=d_model,
                                                      nhead=n_heads,
                                                      dropout=dropout)
        self.transformer = nn.TransformerEncoder(encoder_layer,
                                                 num_layers=num_layers)
        self.classifier  = nn.Linear(d_model, num_classes)

    def forward(self, x):
        # x: (batch, seq_len, n_channels)
        x = self.input_proj(x)           # → (batch, seq_len, d_model)
        x = self.pos_enc(x)
        x = x.permute(1, 0, 2)           # → (seq_len, batch, d_model)
        enc = self.transformer(x)        # same shape
        out = enc[-1]                    # last time-step (batch, d_model)
        return self.classifier(out)      # logits (batch, num_classes)

# ─── 3) Instantiate & train ─────────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerHAR(n_channels=raw_train.shape[2],
                       d_model=64,
                       n_heads=4,
                       num_layers=2,
                       num_classes=len(le.classes_)).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 21):
    model.train()
    total_loss = 0.0
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(Xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch:2d} | Loss: {total_loss/len(train_loader):.4f}")

# ─── 4) Evaluate ─────────────────────────────────────────────────────────────
model.eval()
correct = 0
with torch.no_grad():
    for Xb, yb in test_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        preds = model(Xb).argmax(dim=1)
        correct += (preds == yb).sum().item()
accuracy = correct / len(test_ds)
print(f"\nTransformer HAR Test Accuracy: {accuracy:.4f}")


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch  1 | Loss: 0.8457
Epoch  2 | Loss: 0.3078
Epoch  3 | Loss: 0.2082
Epoch  4 | Loss: 0.1914
Epoch  5 | Loss: 0.1782
Epoch  6 | Loss: 0.1532
Epoch  7 | Loss: 0.1426
Epoch  8 | Loss: 0.1363
Epoch  9 | Loss: 0.1396
Epoch 10 | Loss: 0.1418
Epoch 11 | Loss: 0.1478
Epoch 12 | Loss: 0.1314
Epoch 13 | Loss: 0.1226
Epoch 14 | Loss: 0.1095
Epoch 15 | Loss: 0.1107
Epoch 16 | Loss: 0.1164
Epoch 17 | Loss: 0.1084
Epoch 18 | Loss: 0.1101
Epoch 19 | Loss: 0.1304
Epoch 20 | Loss: 0.1162

Transformer HAR Test Accuracy: 0.8856


# AR

In [ ]:
# ─── 1) Load raw inertial windows (only once) ─────────────────────────────────
import os, pandas as pd, numpy as np
from statsmodels.tsa.ar_model import AutoReg

def load_raw_windows(base_dir="/mnt/data/UCI_HAR_Dataset", split="train"):
    folder   = os.path.join(base_dir, split, "Inertial Signals")
    channels = ["total_acc","body_acc","body_gyro"]
    axes     = ["x","y","z"]
    mats     = []
    for ch in channels:
        for ax in axes:
            fn = f"{ch}_{ax}_{split}.txt"
            path = os.path.join(folder, fn)
            arr  = pd.read_csv(path, delim_whitespace=True, header=None).values
            mats.append(arr[..., np.newaxis])
    return np.concatenate(mats, axis=2)

raw_train = load_raw_windows("UCI_HAR_Dataset","train")  # (7352,128,6)
raw_test  = load_raw_windows("UCI_HAR_Dataset","test")   # (2947,128,6)

# ─── 2) Extract autoregressive (AR) features ──────────────────────────────────
ar_lags = 4
n_windows, _, n_ch = raw_train.shape

def extract_ar_features(X, lags):
    feats = np.zeros((X.shape[0], n_ch * (lags+1)))
    for i in range(X.shape[0]):
        v = []
        for c in range(n_ch):
            model = AutoReg(X[i, :, c], lags=lags, old_names=False).fit()
            v.extend(model.params)   # [intercept, coeff1…coeff_lags]
        feats[i] = v
    return feats

ar_train = extract_ar_features(raw_train, ar_lags)  # (7352,6*5=30)
ar_test  = extract_ar_features(raw_test,  ar_lags)  # (2947,30)

print("AR feature shapes:", ar_train.shape, ar_test.shape)

# ─── 3) Prepare final feature matrices & labels ──────────────────────────────
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(train_df["Activity"])
y_test  = le.transform(     test_df ["Activity"])

# Use your 100 reduced features (replace with your actual list if not the first 100)
feat_cols = [c for c in train_df.columns if c!="Activity"][:100]

X_train = np.hstack([train_df[feat_cols].values, ar_train])  # (7352,100+30=130)
X_test  = np.hstack([ test_df [feat_cols].values, ar_test ])  # (2947,130)

# ─── 4) Train & evaluate an SVM on AR features ───────────────────────────────
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

clf = SVC(kernel="rbf", C=1.0, gamma="scale")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("AR+SVM Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))


<ipython-input-14-f5f733358ee5>:14: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  arr  = pd.read_csv(path, delim_whitespace=True, header=None).values
<ipython-input-14-f5f733358ee5>:14: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  arr  = pd.read_csv(path, delim_whitespace=True, header=None).values
<ipython-input-14-f5f733358ee5>:14: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  arr  = pd.read_csv(path, delim_whitespace=True, header=None).values
<ipython-input-14-f5f733358ee5>:14: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  arr  = pd.read_csv(path, delim_whitespace=True, header=None).values
<ipython

AR feature shapes: (7352, 45) (2947, 45)
AR+SVM Accuracy: 0.9127926705123854
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.90      0.80      0.84       491
          STANDING       0.83      0.92      0.87       532
           WALKING       0.88      0.97      0.92       496
WALKING_DOWNSTAIRS       0.98      0.88      0.93       420
  WALKING_UPSTAIRS       0.91      0.90      0.90       471

          accuracy                           0.91      2947
         macro avg       0.92      0.91      0.91      2947
      weighted avg       0.92      0.91      0.91      2947



# VAE

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# ─── Prerequisites: ensure raw_train/raw_test and train_df/test_df are loaded ──
# raw_train/raw_test: numpy arrays (n_windows, 128, n_channels)
# train_df/test_df: pandas DataFrames with 'Activity' column

# Flatten each window to a vector of size 128*n_channels
n_windows, seq_len, n_ch = raw_train.shape
input_dim = seq_len * n_ch

X_tr_flat = torch.tensor(raw_train.reshape(n_windows, -1), dtype=torch.float32)
X_te_flat = torch.tensor(raw_test.reshape(raw_test.shape[0], -1), dtype=torch.float32)

# ─── 1) Define VAE architecture ───────────────────────────────────────────────
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, latent_dim=32):
        super().__init__()
        # Encoder
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        # Decoder
        self.fc_dec1 = nn.Linear(latent_dim, hidden_dim)
        self.fc_dec2 = nn.Linear(hidden_dim, input_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h = self.relu(self.fc1(x))
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.relu(self.fc_dec1(z))
        return self.sigmoid(self.fc_dec2(h))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon = self.decode(z)
        return recon, mu, logvar

# ─── 2) VAE loss function ─────────────────────────────────────────────────────
def vae_loss(recon_x, x, mu, logvar):
    # Reconstruction loss (MSE)
    recon_loss = nn.MSELoss(reduction='sum')(recon_x, x)
    # KL divergence
    kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kld

# ─── 3) Train VAE ─────────────────────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae = VAE(input_dim=input_dim, hidden_dim=512, latent_dim=32).to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

train_ds = TensorDataset(X_tr_flat)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

vae.train()
epochs = 50
for epoch in range(1, epochs+1):
    total_loss = 0
    for batch in train_loader:
        x = batch[0].to(device)
        optimizer.zero_grad()
        recon, mu, logvar = vae(x)
        loss = vae_loss(recon, x, mu, logvar)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch:2d} | VAE Loss: {total_loss/len(train_loader.dataset):.4f}")

# ─── 4) Extract latent features ───────────────────────────────────────────────
vae.eval()
with torch.no_grad():
    mu_train, _ = vae.encode(X_tr_flat.to(device))
    mu_test,  _ = vae.encode(X_te_flat.to(device))
latent_train = mu_train.cpu().numpy()
latent_test  = mu_test.cpu().numpy()

# ─── 5) Classify using SVM on latent features ────────────────────────────────
le = LabelEncoder()
y_train = le.fit_transform(train_df['Activity'])
y_test  = le.transform(test_df['Activity'])

clf = SVC(kernel='rbf', C=1.0, gamma='scale')
clf.fit(latent_train, y_train)
y_pred = clf.predict(latent_test)
print("VAE+SVM Accuracy:", accuracy_score(y_test, y_pred))


Epoch  1 | VAE Loss: 104.2549
Epoch  2 | VAE Loss: 80.6574
Epoch  3 | VAE Loss: 79.3545
Epoch  4 | VAE Loss: 78.8954
Epoch  5 | VAE Loss: 78.6769
Epoch  6 | VAE Loss: 78.2508
Epoch  7 | VAE Loss: 77.9099
Epoch  8 | VAE Loss: 77.6425
Epoch  9 | VAE Loss: 77.2259
Epoch 10 | VAE Loss: 76.5349
Epoch 11 | VAE Loss: 75.6371
Epoch 12 | VAE Loss: 75.3261
Epoch 13 | VAE Loss: 74.9036
Epoch 14 | VAE Loss: 74.4254
Epoch 15 | VAE Loss: 74.0651
Epoch 16 | VAE Loss: 73.9405
Epoch 17 | VAE Loss: 73.4677
Epoch 18 | VAE Loss: 73.3555
Epoch 19 | VAE Loss: 73.1744
Epoch 20 | VAE Loss: 72.9881
Epoch 21 | VAE Loss: 72.8220
Epoch 22 | VAE Loss: 72.6397
Epoch 23 | VAE Loss: 72.6108
Epoch 24 | VAE Loss: 72.4595
Epoch 25 | VAE Loss: 72.4013
Epoch 26 | VAE Loss: 72.2302
Epoch 27 | VAE Loss: 72.1515
Epoch 28 | VAE Loss: 72.3928
Epoch 29 | VAE Loss: 72.0187
Epoch 30 | VAE Loss: 72.1622
Epoch 31 | VAE Loss: 72.0823
Epoch 32 | VAE Loss: 71.7562
Epoch 33 | VAE Loss: 71.7412
Epoch 34 | VAE Loss: 71.5912
Epoch 35 | VA